# Detekcia meteorov na rádiových spektrogramoch založená na konvolučných neurónových sieťach

## Príprava pracovného prostredia pre spracovanie dát

Inštalácia knižníc potrebných pre prácu.

In [ ]:
!pip install opencv-python 
!pip install python_utils
!pip install python-csv
!pip install pdf2image
!pip install pandas
!pip install numpy
!pip install python-math
!pip install jsonlib
!pip install os-sys
!pip install Pillow
!pip install temp

Import potrebných knižníc.

In [ ]:
import os
import io
import tempfile
import zipfile,fnmatch
import csv
import pandas as pd
import numpy as np
import json
import math 
from PIL import Image, ImageDraw
import operator
from skimage.io import imread
import matplotlib.pyplot as plt
import seaborn as sns
import sys
%matplotlib inline
sns.set()
import statistics as st
import matplotlib.patches as patches

## Príprava dát

Načítanie dát do premennej.

In [ ]:
data=pd.read_csv("ozveny-classifications_3.csv")
#data=pd.read_csv("ozveny-dokoncenie-classifications.csv")

Odstránenie nepotrebných stĺpcov.

In [ ]:
data=data.drop('workflow_name',axis=1).drop('workflow_id',axis=1).drop('workflow_version',axis=1).drop('user_ip',axis=1).drop('created_at',axis=1).drop('gold_standard',axis=1).drop('expert',axis=1).drop('user_name',axis=1).drop('user_id',axis=1).drop('classification_id',axis=1)

Uloženie cesty k výstupu do premennej output.

In [ ]:
output='output.csv'

Uloženie cesty k dátam do premennej rootPath.

In [ ]:
rootPath=r"data"

Odstránenie chybných riadkov v dátach.

In [ ]:
i=0
while(i<len(data)):
    je_ozvena=json.loads(data.iloc[i,1])[0]["value"]
    je_none=json.loads(data.iloc[i,1])[0]["value"]
    if je_ozvena=='Áno':
        dlzka=len(json.loads(data.iloc[i,1])[1]["value"])
        if dlzka==0:
            data=data.drop(data.index[i])
    if je_none==None:
        data=data.drop(data.index[i])
    i+=1
    

Odstránené riadky iba zo súboru ozveny-classifications_3.csv je potrebné ostrániť aj v dátach.

In [ ]:
data=data.drop(data.index[492]) 
data=data.drop(data.index[519])
data=data.drop(data.index[1774])
data=data.drop(data.index[2452])

Kód pre triedenie dát od duplicitných.

In [ ]:
output = "filename.csv"
with open(output, mode='w') as csv_file:
    columns = ["filename"]
    write = csv.DictWriter(csv_file, fieldnames=columns)
    write.writeheader()
    i = 0
    while (i < len(data)):
        file_name = json.loads(data.iloc[i, 2])[str(data.iloc[i, 3])]["Filename"]
        write.writerow(
            {'filename': file_name})
        i+=1
    print(len(data))

## Vykreslenie anotácií a vytvorenie výstupnej tabuľky
### Vstupy
data: premenná obsahujúca cestu k súborom ozveny-classifications_3.csv a ozveny-dokoncenie-classifications.csv <br>
output: premenná obsahujúca cestu k výstupnému súboru <br>
rootPath: premenná obsahujúca cestu k dátam bez anotácií

In [ ]:
with open(output, mode='w') as csv_file:
    stlpce=["event","subject_id","x","y","sirka","vyska","headecho","typ_ozveny"]
    zapis=csv.DictWriter(csv_file,fieldnames=stlpce)
    zapis.writeheader()
    i=0
    while (i<len(data)):
        je_ozvena=json.loads(data.iloc[i,1])[0]["value"]
        if je_ozvena=="Áno":
            anotacia=json.loads(data.iloc[i,1])[1]["value"]
            event=json.loads(data.iloc[i,2])[str(data.iloc[i,3])]["Filename"]
            subject_id=json.loads(str(data.iloc[i,3]))
            x=json.loads(data.iloc[i,1])[1]["value"][0]["x"]
            y=json.loads(data.iloc[i,1])[1]["value"][0]["y"]
            sirka=json.loads(data.iloc[i,1])[1]["value"][0]["width"]
            vyska=json.loads(data.iloc[i,1])[1]["value"][0]["height"]
            headecho=json.loads(data.iloc[i,1])[1]["value"][0]["details"][0]["value"]
            if headecho==1:
                headecho="nie"
            else:
                headecho="ano"
            typ_ozveny=json.loads(data.iloc[i,1])[1]["value"][0]["tool_label"]
            zapis.writerow({'event': event,'subject_id': subject_id,'x': x,'y':y,'sirka':sirka,'vyska':vyska,'headecho':headecho,'typ_ozveny':typ_ozveny})
            for root, dirs, files in os.walk(rootPath):
                for file in files:
                    if file == event:
                        im = Image.open(os.path.join(root, "%s" % (file)))
                        im1 = ImageDraw.Draw(im)
                        if (typ_ozveny==('Ozvena O (dlhá)' or 'Ozvena O')):
                            shape=[(x, y), (x, y+vyska)]                 
                            im1.line(shape, fill="white", width=2)
                            shape=[(x, y+vyska),(x+sirka,y+vyska) ]
                            im1.line(shape, fill="white", width=2)
                            shape=[(x, y), (x+sirka, y)]
                            im1.line(shape, fill="white", width=2)
                            shape=[(x+sirka, y),(x+sirka,y+vyska) ]
                            im1.line(shape, fill="white", width=2)
                            print("File %s done." % (event))
                            im.save(os.path.join('anotacie', "1b%s" % (file)))
                        else:
                            shape=[(x, y), (x, y+vyska)]
                            im1.line(shape,fill="red", width=2)
                            shape=[(x, y+vyska),(x+sirka,y+vyska) ]
                            im1.line(shape,fill="red", width=2)
                            shape=[(x, y), (x+sirka, y)]
                            im1.line(shape,fill="red", width=2)
                            shape=[(x+sirka, y),(x+sirka,y+vyska) ]
                            im1.line(shape,fill="red", width=2)
                            print("File %s done." % (event))
                            im.save(os.path.join('anotacie', "1%s" % (file)))  
            if (len(anotacia)>1):
                event=json.loads(data.iloc[i,2])[str(data.iloc[i,3])]["Filename"]
                subject_id=json.loads(str(data.iloc[i,3]))
                x=json.loads(data.iloc[i,1])[1]["value"][1]["x"]
                y=json.loads(data.iloc[i,1])[1]["value"][1]["y"]
                sirka=json.loads(data.iloc[i,1])[1]["value"][1]["width"]
                vyska=json.loads(data.iloc[i,1])[1]["value"][1]["height"]
                headecho=json.loads(data.iloc[i,1])[1]["value"][1]["details"][0]["value"]
                if headecho==1:
                    headecho="nie"
                else:
                    headecho="ano"
                typ_ozveny=json.loads(data.iloc[i,1])[1]["value"][1]["tool_label"]
                zapis.writerow({'event': event,'subject_id': subject_id,'x': x,'y':y,'sirka':sirka,'vyska':vyska,'headecho':headecho,'typ_ozveny':typ_ozveny})
                for root, dirs, files in os.walk(rootPath):
                    for file in files:
                        if file == event:
                            im = Image.open(os.path.join(root, "%s" % (file)))
                            im1 = ImageDraw.Draw(im)
                            if (typ_ozveny==('Ozvena O (dlhá)' or 'Ozvena O')):
                                shape=[(x, y), (x, y+vyska)]                 
                                im1.line(shape, fill="white", width=2)
                                shape=[(x, y+vyska),(x+sirka,y+vyska) ]
                                im1.line(shape, fill="white", width=2)
                                shape=[(x, y), (x+sirka, y)]
                                im1.line(shape, fill="white", width=2)
                                shape=[(x+sirka, y),(x+sirka,y+vyska) ]
                                im1.line(shape, fill="white", width=2)
                                print("File %s done." % (event))
                                im.save(os.path.join('anotacie', "2b%s" % (file))) 
                            else:
                                shape=[(x, y), (x, y+vyska)]
                                im1.line(shape,fill="red", width=2)
                                shape=[(x, y+vyska),(x+sirka,y+vyska) ]
                                im1.line(shape,fill="red", width=2)
                                shape=[(x, y), (x+sirka, y)]
                                im1.line(shape,fill="red", width=2)
                                shape=[(x+sirka, y),(x+sirka,y+vyska) ]
                                im1.line(shape,fill="red", width=2)
                                print("File %s done." % (event))
                                im.save(os.path.join('anotacie', "2%s" % (file))) 
                if (len(anotacia)>2):
                    event=json.loads(data.iloc[i,2])[str(data.iloc[i,3])]["Filename"]
                    subject_id=json.loads(str(data.iloc[i,3]))
                    x=json.loads(data.iloc[i,1])[1]["value"][2]["x"]
                    y=json.loads(data.iloc[i,1])[1]["value"][2]["y"]
                    sirka=json.loads(data.iloc[i,1])[1]["value"][2]["width"]
                    vyska=json.loads(data.iloc[i,1])[1]["value"][2]["height"]
                    headecho=json.loads(data.iloc[i,1])[1]["value"][2]["details"][0]["value"]
                    if headecho==1:
                        headecho="nie"
                    else:
                        headecho="ano"
                    typ_ozveny=json.loads(data.iloc[i,1])[1]["value"][2]["tool_label"]
                    zapis.writerow({'event': event,'subject_id': subject_id,'x': x,'y':y,'sirka':sirka,'vyska':vyska,'headecho':headecho,'typ_ozveny':typ_ozveny})
                    for root, dirs, files in os.walk(rootPath):
                        for file in files:
                            if file == event:
                                im = Image.open(os.path.join(root, "%s" % (file)))
                                im1 = ImageDraw.Draw(im)
                                if (typ_ozveny==('Ozvena O (dlhá)' or 'Ozvena O')):
                                    shape=[(x, y), (x, y+vyska)]                 
                                    im1.line(shape, fill="white", width=2)
                                    shape=[(x, y+vyska),(x+sirka,y+vyska) ]
                                    im1.line(shape, fill="white", width=2)
                                    shape=[(x, y), (x+sirka, y)]
                                    im1.line(shape, fill="white", width=2)
                                    shape=[(x+sirka, y),(x+sirka,y+vyska) ]
                                    im1.line(shape, fill="white", width=2)
                                    print("File %s done." % (event))
                                    im.save(os.path.join('anotacie', "3b%s" % (file)))
                                else:
                                    shape=[(x, y), (x, y+vyska)]
                                    im1.line(shape,fill="red", width=2)
                                    shape=[(x, y+vyska),(x+sirka,y+vyska) ]
                                    im1.line(shape,fill="red", width=2)
                                    shape=[(x, y), (x+sirka, y)]
                                    im1.line(shape,fill="red", width=2)
                                    shape=[(x+sirka, y),(x+sirka,y+vyska) ]
                                    im1.line(shape,fill="red", width=2)
                                    print("File %s done." % (event))
                                    im.save(os.path.join('anotacie', "3%s" % (file)))
                             
                    if (len(anotacia)>3):
                        event=json.loads(data.iloc[i,2])[str(data.iloc[i,3])]["Filename"]
                        subject_id=json.loads(str(data.iloc[i,3]))
                        x=json.loads(data.iloc[i,1])[1]["value"][3]["x"]
                        y=json.loads(data.iloc[i,1])[1]["value"][3]["y"]
                        sirka=json.loads(data.iloc[i,1])[1]["value"][3]["width"]
                        vyska=json.loads(data.iloc[i,1])[1]["value"][3]["height"]
                        headecho=json.loads(data.iloc[i,1])[1]["value"][3]["details"][0]["value"]
                        if headecho==1:
                            headecho="nie"
                        else:
                            headecho="ano"
                        typ_ozveny=json.loads(data.iloc[i,1])[1]["value"][3]["tool_label"]
                        zapis.writerow({'event': event,'subject_id': subject_id,'x': x,'y':y,'sirka':sirka,'vyska':vyska,'headecho':headecho,'typ_ozveny':typ_ozveny})
                        for root, dirs, files in os.walk(rootPath):
                            for file in files:
                                if file == event:
                                    im = Image.open(os.path.join(root, "%s" % (file)))
                                    im1 = ImageDraw.Draw(im)
                                    if (typ_ozveny==('Ozvena O (dlhá)' or 'Ozvena O')):
                                        shape=[(x, y), (x, y+vyska)]                 
                                        im1.line(shape, fill="white", width=2)
                                        shape=[(x, y+vyska),(x+sirka,y+vyska) ]
                                        im1.line(shape, fill="white", width=2)
                                        shape=[(x, y), (x+sirka, y)]
                                        im1.line(shape, fill="white", width=2)
                                        shape=[(x+sirka, y),(x+sirka,y+vyska) ]
                                        im1.line(shape, fill="white", width=2)
                                        print("File %s done." % (event))
                                        im.save(os.path.join('anotacie', "4b%s" % (file)))
                                    else:
                                        shape=[(x, y), (x, y+vyska)]
                                        im1.line(shape,fill="red", width=2)
                                        shape=[(x, y+vyska),(x+sirka,y+vyska) ]
                                        im1.line(shape,fill="red", width=2)
                                        shape=[(x, y), (x+sirka, y)]
                                        im1.line(shape,fill="red", width=2)
                                        shape=[(x+sirka, y),(x+sirka,y+vyska) ]
                                        im1.line(shape,fill="red", width=2)
                                        print("File %s done." % (event))
                                        im.save(os.path.join('anotacie', "4%s" % (file)))
                  
        i+=1
    

## Vytvorenie textového súboru so súradnicami pre neurónovú sieť YOLO
### Vstupy
data: premenná obsahujúca cestu k súborom ozveny-classifications_3.csv a ozveny-dokoncenie-classifications.csv <br>
rootPath: premenná obsahujúca cestu k dátam bez anotácií <br>
textFile2: cesta k súboru, do ktorého sa dáta uložia

In [ ]:
textFile2=os.getcwd()+"/txt_ok2"
i=0
while (i<len(data)):
    je_ozvena2=json.loads(data.iloc[i,1])[0]["value"]
    if je_ozvena2=="Áno":
        anotacie2=json.loads(data.iloc[i,1])[1]["value"]     
        event2=json.loads(data.iloc[i,2])[str(data.iloc[i,3])]["Filename"].split(".")     
        nazovjpg=json.loads(data.iloc[i,2])[str(data.iloc[i,3])]["Filename"]
        pocet2=0
        with open(os.path.join(textFile2, "%s.txt" % (event2[0])), mode='w') as csv_file:
            while(pocet2<len(anotacie2)):
                subject_id2=json.loads(str(data.iloc[i,3]))
                x2=json.loads(data.iloc[i,1])[1]["value"][pocet2]["x"]
                y2=json.loads(data.iloc[i,1])[1]["value"][pocet2]["y"]
                width2=json.loads(data.iloc[i,1])[1]["value"][pocet2]["width"]
                height2=json.loads(data.iloc[i,1])[1]["value"][pocet2]["height"]
                typ_ozveny2=json.loads(data.iloc[i,1])[1]["value"][pocet2]["tool_label"]
                print(typ_ozveny2)
                print(i,event2)
                headecho2=json.loads(data.iloc[i,1])[1]["value"][pocet2]["details"][0]["value"]
                print(headecho2)
                obrazok=cv2.imread("pozitivne/"+nazovjpg)  
                vyska=obrazok.shape[0]
                print(vyska)
                if(headecho2==0):       
                    if((typ_ozveny2=="Ozvena U.") or (typ_ozveny2=="Ozvena U") or (typ_ozveny2=="Ozvena U (krátka)") or (typ_ozveny2=="Ozvena U (krátka).")):       # krátka s headechom
                        label=1
                    elif((typ_ozveny2=="Ozvena O (dlhá)") or (typ_ozveny2=="Ozvena O") or (typ_ozveny2=="Ozvena O.") or (typ_ozveny2=="Ozvena O (dlhá).")):         # dlhá s headechom
                        label=3
                elif(headecho2==1): 
                    if((typ_ozveny2=="Ozvena U.") or (typ_ozveny2=="Ozvena U") or (typ_ozveny2=="Ozvena U (krátka)") or (typ_ozveny2=="Ozvena U (krátka).")):       # krátka bez headecha
                        label=0
                    elif((typ_ozveny2=="Ozvena O (dlhá)") or (typ_ozveny2=="Ozvena O") or (typ_ozveny2=="Ozvena O.") or (typ_ozveny2=="Ozvena O (dlhá).")):         # dlhá bez headecha
                        label=2
                print(label)
                fieldnames1=[label,round(((x2+(width2/2))/1218),5),round(((y2+(height2/2))/vyska),5),round((width2/1218),5),round((height2/vyska),5)]
                writer2=csv.DictWriter(csv_file,fieldnames=fieldnames1, delimiter=' ')
                writer2.writeheader()
                writer2.writerow({'label': label,'x': round(((x2+(width2/2))/1216),5),'y':round(((y2+(height2/2))/686),5),'width':round((width2/1216),5),'height':round((height2/686),5)})
                print("Just made normalized txt file for %s" % (event2[0]))
                pocet2+=1
    i+=1
    
